In [2]:
import pandas as pd

# Load each file
products_df = pd.read_csv(r"C:\Users\mgag2\OneDrive\Documentos\Data\CRM+Sales+Opportunities\products.csv")
accounts_df = pd.read_csv(r"C:\Users\mgag2\OneDrive\Documentos\Data\CRM+Sales+Opportunities\accounts.csv")
sales_teams_df = pd.read_csv(r"C:\Users\mgag2\OneDrive\Documentos\Data\CRM+Sales+Opportunities\sales_teams.csv")
sales_pipeline_df = pd.read_csv(r"C:\Users\mgag2\OneDrive\Documentos\Data\CRM+Sales+Opportunities\sales_pipeline.csv")

# See only headers
print(sales_pipeline_df.columns.tolist())
print(accounts_df.columns.tolist())
print(products_df.columns.tolist())
print(sales_teams_df.columns.tolist())

['opportunity_id', 'sales_agent', 'product', 'account', 'deal_stage', 'engage_date', 'close_date', 'close_value']
['account', 'sector', 'year_established', 'revenue', 'employees', 'office_location', 'subsidiary_of']
['product', 'series', 'sales_price']
['sales_agent', 'manager', 'regional_office']


In [8]:
# See data
print("Sales Pipeline sample")
print("")
print(sales_pipeline_df.head())
print("")
print("Products sample")
print("")
print(products_df.head())
print("")
print("Accounts sample")
print("")
print(accounts_df.head())
print("")
print("Sales Team sample")
print("")
print(sales_teams_df.head())

Sales Pipeline sample

  opportunity_id      sales_agent         product  account deal_stage  \
0       1C1I7A6R      Moses Frase  GTX Plus Basic  Cancity        Won   
1       Z063OYW0  Darcel Schlecht          GTXPro    Isdom        Won   
2       EC4QE1BX  Darcel Schlecht      MG Special  Cancity        Won   
3       MV1LWRNH      Moses Frase       GTX Basic  Codehow        Won   
4       PE84CX4O        Zane Levy       GTX Basic   Hatfan        Won   

  engage_date  close_date  close_value  
0  2016-10-20  2017-03-01       1054.0  
1  2016-10-25  2017-03-11       4514.0  
2  2016-10-25  2017-03-07         50.0  
3  2016-10-25  2017-03-09        588.0  
4  2016-10-25  2017-03-02        517.0  

Products sample

        product series  sales_price
0     GTX Basic    GTX          550
1       GTX Pro    GTX         4821
2    MG Special     MG           55
3   MG Advanced     MG         3393
4  GTX Plus Pro    GTX         5482

Accounts sample

            account     sector  year_est

In [10]:
#Check data with custom function based on describe
from custom_summary import custom_describe
custom_describe(sales_pipeline_df)

,Column,Type,Count,Unique,Missing,Min,Max,Average
0,opportunity_id,object,8800,8800,0,NaN,NaN,NaN
1,sales_agent,object,8800,30,0,NaN,NaN,NaN
2,product,object,8800,7,0,NaN,NaN,NaN
3,account,object,7375,85,1425,NaN,NaN,NaN
4,deal_stage,object,8800,4,0,NaN,NaN,NaN
5,engage_date,object,8300,421,500,NaN,NaN,NaN
6,close_date,object,6711,306,2089,NaN,NaN,NaN
7,close_value,float64,6711,2051,2089,0.0,30288.0,1490.915512


#### The data is mostly clean, but there is a couple of things we need to handle.

1. There's a product name that was incorrectly added to the pipeline file.
2. There are 1425 missing values for account in the pipeline file.

In [14]:
# Fixing product name in sales_pipeline
# Replace only in the 'product' column
sales_pipeline_df['product'] = sales_pipeline_df['product'].replace("GTXPro", "GTX Pro")
print(sales_pipeline_df['product'].sample(10))

7361           GTX Pro
2258        MG Special
2431       MG Advanced
3462      GTX Plus Pro
7651       MG Advanced
6213    GTX Plus Basic
1664       MG Advanced
3194         GTX Basic
5695    GTX Plus Basic
7515    GTX Plus Basic
Name: product, dtype: object


In [16]:
# Merge sales pipeline with products
sales_with_products = pd.merge(sales_pipeline_df, products_df, on="product", how="left")

# Ensure all rows are matched
num_unmatched = sales_with_products['product'].isna().sum()
if num_unmatched == 0:
    print("All products matched!")
else:
    print(f"There are {num_unmatched} unmatched products.")

All products matched!


In [18]:
# Merge result with accounts
sales_with_accounts = pd.merge(sales_with_products, accounts_df, on="account", how="left")

num_unmatched = sales_with_accounts['account'].isna().sum()
if num_unmatched == 0:
    print("All accounts matched!")
else:
    print(f"There are {num_unmatched} unmatched accounts.")

There are 1425 unmatched accounts.


#### As we expected, 1425 lines didn't match due to missing values.  This do not represent an issue because we won't be using that information this time.

In [20]:
# Merge result with sales teams
sales_df = pd.merge(sales_with_accounts, sales_teams_df, on="sales_agent", how="left")
num_unmatched = sales_df['sales_agent'].isna().sum()
if num_unmatched == 0:
    print("All sales agents matched!")
else:
    print(f"There are {num_unmatched} unmatched sales agents.")

All sales agents matched!


In [22]:
print(sales_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8800 entries, 0 to 8799
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   opportunity_id    8800 non-null   object 
 1   sales_agent       8800 non-null   object 
 2   product           8800 non-null   object 
 3   account           7375 non-null   object 
 4   deal_stage        8800 non-null   object 
 5   engage_date       8300 non-null   object 
 6   close_date        6711 non-null   object 
 7   close_value       6711 non-null   float64
 8   series            8800 non-null   object 
 9   sales_price       8800 non-null   int64  
 10  sector            7375 non-null   object 
 11  year_established  7375 non-null   float64
 12  revenue           7375 non-null   float64
 13  employees         7375 non-null   float64
 14  office_location   7375 non-null   object 
 15  subsidiary_of     1292 non-null   object 
 16  manager           8800 non-null   object 


In [24]:
print(sales_df['deal_stage'].unique())

['Won' 'Engaging' 'Lost' 'Prospecting']


#### Creating a new column named 'main_stage' to rename the Won and Lost stages.

In [26]:
# Function to determine main_stage
def determine_main_stage(deal_stage):
    if deal_stage == 'Won':
        return 'Closed Won'
    elif deal_stage == 'Lost':
        return 'Closed Lost'
    else:
        return deal_stage

# Apply the function to create next_stage column
sales_df['main_stage'] = sales_df['deal_stage'].apply(determine_main_stage)
print(sales_df.columns.tolist())

['opportunity_id', 'sales_agent', 'product', 'account', 'deal_stage', 'engage_date', 'close_date', 'close_value', 'series', 'sales_price', 'sector', 'year_established', 'revenue', 'employees', 'office_location', 'subsidiary_of', 'manager', 'regional_office', 'main_stage']


In [28]:
print(sales_df['main_stage'].unique())

['Closed Won' 'Engaging' 'Closed Lost' 'Prospecting']


#### This dataframe will be used in other notebooks, saving it

In [30]:
%store sales_df

Stored 'sales_df' (DataFrame)
